# Initialization

Test notebook for the DataHandlers. Test the CMAPSS DataHandler

First we import the necessary packages and create the global variables.

In [1]:
import numpy as np
import math
import tensorflow as tf

from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D, LSTM
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

import CMAPSAuxFunctions

from numpy.random import seed
from tensorflow import set_random_seed


/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


# Define different types of Keras models

In [2]:
l2_lambda_regularization = 0
#l1_lambda_regularization = 0.2

def RULmodel_LSTM(input_shape):
    """Define the RNN model"""
    
    #Create a sequential model
    model = Sequential()
    #model.add(Masking(mask_value=0, imput))
    #model.add(LSTM(input_shape=input_shape, units=100, return_sequences=True, name='lstm1')))
    model.add(LSTM(input_shape=input_shape, units=20, return_sequences=False, name='lstm2'))
    model.add(Dense(10, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model
    

def RULmodel_SN_5(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_shape=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(20, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), 
                    name='fc2'))
    model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='out'))
    
    return model

def test_model(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', 
                    kernel_initializer=keras.initializers.glorot_normal(seed=0), 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', 
                    kernel_initializer=keras.initializers.glorot_normal(seed=0), 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='out'))
    
    return model

# Tensorflow Models

In [3]:
def create_placeholders(input_shape, output_shape):
    
    X = tf.placeholder(tf.float32, shape=(None,input_shape), name="X")
    y = tf.placeholder(tf.float32, shape=None, name="y")
    
    return X, y

#regularizers.l2(l2_lambda_regularization)
def tf_model(X):
    
    l2_lambda_regularization = 0.20
    l1_lambda_regularization = 0.10
    
    A1 = tf.layers.dense(X, 20, activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False), 
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="fc1")
    A2 = tf.layers.dense(A1, 20, activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="fc2")
    y = tf.layers.dense(A2, 1, activation=None, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                        kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="out")
    
    return y

In [4]:
def print_model_weights(model):

    for layer in model.layers:
        weights = layer.get_weights() # list of numpy arrays
        
        for weight in weights:
        
            print(weight.shape)
            print(weight)

# Create the Data Handler

In [5]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

# Create Tunable Model

In [7]:
min_max_scaler = MinMaxScaler()
#min_max_scaler = None

In [8]:
def get_compiled_model(shape, model_type='ann'):
    
    K.clear_session()  #Clear the previous tensorflow graph
    
    #To test the model without randomness

    
    #Shared parameters for the models
    optimizer = Adam(lr=0.001,beta_1=0.5)
    
    #optimizer = SGD(lr=0.001, momentum=0.0, decay=0.0, nesterov=False)
    
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models

    if model_type=='ann':
        model = test_model(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    elif model_type=='lstm':
        model = RULmodel_LSTM(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    else:
        pass

    return model


def tf_compiled_model(input_shape, output_shape):
    
    tf.reset_default_graph()
    X, y = create_placeholders(input_shape, output_shape)
    
    y_pred = tf_model(X)
    cost = tf.losses.mean_squared_error(y, y_pred)
    reg_cost = tf.losses.get_regularization_loss()
    total_cost = cost + reg_cost
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001,beta1=0.5).minimize(total_cost)
    
    return {'X_placeholder':X, 'y_placeholder':y, 'y_pred':y_pred, 'cost':cost, 'total_cost':total_cost, 'optimizer':optimizer}

In [9]:
num_features = len(selected_features)

shape = len(selected_features)*window_size
model = get_compiled_model(shape, model_type='ann')
tModel = SequenceTunableModelRegression('ModelRUL_SN_1', model, lib_type='keras', data_handler=dHandler_cmaps)

#shape = (window_size, num_features)
#model = get_compiled_model(shape, model_type='lstm')
#tModel = SequenceTunableModelRegression('ModelRUL_LSTM_1', model, lib_type='keras', data_handler=dHandler_cmaps)
#tModel.data_handler.data_scaler = min_max_scaler
#tModel.data_scaler = min_max_scaler

# Load Data

In [10]:
tModel.data_handler.sequence_length = 25
#tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
tModel.data_handler.sequence_stride = 1
tModel.data_handler.max_rul = 130

In [11]:
#Load Non sequenced data (unroll sequence into a single feature vector)

tModel.data_handler.data_scaler = None
tModel.data_scaler = min_max_scaler

tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
#tModel.data_handler.print_data()
tModel.print_data()

Loading data for dataset 1 with window_size of 25, stride of 1 and maxRUL of 130. Cros-Validation ratio 0
Loading data from file and computing dataframes
Printing shapes

Training data (X, y)
(18231, 350)
(18231, 1)
Testing data (X, y)
(100, 350)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[0.20469799 0.47723785 0.36008634 ... 0.36363636 0.62790698 0.75063184]
 [0.31543624 0.5314578  0.40992936 ... 0.45454545 0.55813953 0.73111486]
 [0.38255034 0.43350384 0.43072998 ... 0.36363636 0.65891473 0.77646728]
 [0.38255034 0.30051151 0.38500785 ... 0.09090909 0.6744186  0.54745858]
 [0.38926174 0.30204604 0.47036892 ... 0.36363636 0.6124031  0.65360854]]
[[130.]
 [130.]
 [130.]
 [130.]
 [130.]]
Testing data (X, y)
[[0.30201342 0.31457801 0.44328885 ... 0.36363636 0.51937984 0.64729009]
 [0.7114094  0.48925831 0.4405416  ... 0.18181818 0.51937984 0.51614715]
 [0.73154362 0.47237852 0.54238619 ... 0.54545455 0.6124031  0.53327717]
 [0.4261745  0.56726343 0.52982732 ... 0.54545455 

In [ ]:
# #Load sequenced data (do not unroll sequence into a single feature vector)

# tModel.data_handler.data_scaler = min_max_scaler
# tModel.data_scaler = None

# tModel.load_data(unroll=False, verbose=1, cross_validation_ratio=0)
# #tModel.data_handler.print_data()
# tModel.print_data()

In [ ]:
#tModel.data_handler.data_scaler = min_max_scaler
#tModel.data_scaler = None

# Train the model and test some Tunable Model functionalities

In [ ]:
nFeatures = len(selected_features)

#lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

shape = len(selected_features)*tModel.data_handler.sequence_length
print(shape)

#modelRULSN = RULmodel_SN_5(shape)
#modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

#model = get_compiled_model(shape, model_type='ann')
#tModel.change_model('ModelRUL_SN_1', model, 'keras')

model = tf_compiled_model(shape, 1)
tModel.change_model('ModelRUL_SN_1', model, 'tensorflow')

#tModel.print_data()

#shape = (window_size, num_features)
#model = get_compiled_model(shape, model_type='lstm')
#tModel.change_model('ModelRUL_RNN_1', model, 'keras')

#tModel.data_handler.data_scaler = min_max_scaler
#tModel.data_scaler = min_max_scaler

#print("Printing model weights")
#print_model_weights(tModel.model)

tModel.epochs = 200

sess = tf.Session()

tModel.train_model(tf_session=sess, verbose=1)

In [ ]:
tModel.evaluate_model(['rhs', 'rmse'], round=2, tf_session=sess)
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_test)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, np.ravel(tModel.y_predicted), tModel.y_predicted_rounded, np.ravel(tModel.y_test)):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))